In [8]:
import sys  
import os  
import pandas as pd  

# sys.path.insert(0, '..')
from AT_funs import *


filename_bm = 'FTSEMIB.MI.xlsx'
bm_df = read_xlsx(filename_bm)
bm_name = filename_bm.replace('xlsx', '')

# files = [file for file in os.listdir('data') if file.endswith(".xlsx") and file != "FTSEMIB.MI.xlsx" and file != "marginabili.xlsx" and file != "sectors.xlsx" and file != "output_signals.xlsx" and file != "PTF.xlsx", and file == "A2A.MI.xlsx"]
# print(len(files))

exclude_files = {"FTSEMIB.MI.xlsx", "marginabili.xlsx", "sectors.xlsx", "output_signals.xlsx", "PTF.xlsx"}
# include_files = {"A2A.MI.xlsx"}
# files = [file for file in os.listdir('data') if file.endswith(".xlsx") and file not in exclude_files and file in include_files]
files = [file for file in os.listdir() if file.endswith(".xlsx") and file not in exclude_files]
print(len(files))

window_bo = 100
fast = 20
slow = 50
st = 50
mt = 100
lt = 150
lvl = 3
bm_col = 'close'
dgt = 5
window_exp = 100
starting_capital = 100000
lot = 100
mn = -0.0025
mx = -0.0075
# avg = (mn + mx) / 2
tolerance= -0.1
equal_weight = 0.05
span= 5
fx = 1

# Read all xlsx files
dfs = []
last_row_dfs = [None] * len(files)
failed = []
signal_df = pd.DataFrame()
signal_list = []

regime_df = pd.DataFrame()
last_row_df = pd.DataFrame()
signal_df = pd.DataFrame()
dfs_list = []
last_row_list = []
failed = []


for file in files[:1]:
    file_path = os.path.join("", file)
    df = pd.read_excel(file_path)
    if df.shape[0] >= 1000:
        dfs.append(df)

print('letti tutti i files')

416
letti tutti i files


In [9]:
def detect_regime(df, bm_df):
    
    _o,_h,_l,_c = lower_upper_OHLC(df,relative = False)
    rhs = ['hi1', 'lo1','hi2', 'lo2', 'hi3', 'lo3']
    rt_hi,rt_lo,_hi,_lo,shi,slo = [rhs[h] for h in range(len(rhs))]
    bm_col = 'close'
    dgt = 5
    df= relative(df,_o,_h,_l,_c, bm_df, bm_col, dgt= dgt, rebase=True)
    
    params = ['2014-12-31', None, 63, 0.05, 0.05, 1.5, 2,5,2.5,3]
    start,end,vlty_n,dist_pct,retrace_pct,threshold,dgt,d_vol,r_vol,lvl= [params[h] for h in range(len(params))]

    swing_val = ['rg','lo1','hi1','lo3','hi3','clg','flr','rg_ch']
    rg,rt_lo,rt_hi,slo,shi,clg,flr,rg_ch = [swing_val[s] for s in range(len(swing_val))]   

    for a in np.arange(0,2):
        df = round(historical_swings(df,_o,_h,_l,_c, dist= None, hurdle= None),dgt)
        df = cleanup_latest_swing(df,shi,slo,rt_hi,rt_lo)
        ud, bs, bs_dt, _rt, _swg, hh_ll, hh_ll_dt = latest_swing_variables(df,shi,slo,rt_hi,rt_lo,_h,_l, _c)
        vlty = round(average_true_range(df,_h,_l,_c, n= vlty_n)[hh_ll_dt],dgt)
        dist_vol = d_vol * vlty
        _sign = test_distance(ud,bs, hh_ll, dist_vol, dist_pct)
        df = retest_swing(df, _sign, _rt, hh_ll_dt, hh_ll, _c, _swg)
        retrace_vol = r_vol * vlty
        df = retracement_swing(df, _sign, _swg, _c, hh_ll_dt, hh_ll, vlty, retrace_vol, retrace_pct)
        stdev = df[_c].rolling(vlty_n).std(ddof=0)
        df = regime_floor_ceiling(df,_h,_l,_c,slo, shi,flr,clg,rg,rg_ch,stdev,threshold)    

        _o,_h,_l,_c = lower_upper_OHLC(df,relative = True)
        rswing_val = ['rrg','rl1','rh1','rl3','rh3','rclg','rflr','rrg_ch']
        rg,rt_lo,rt_hi,slo,shi,clg,flr,rg_ch = [rswing_val[s] for s in range(len(rswing_val))]
    
    return df

In [18]:
ohlc = ['open','high','low','close']
_o,_h,_l,_c = [ohlc[h] for h in range(len(ohlc))]
i = 0
ticker = dfs[i]['ticker'][0]
dfs = detect_regime(dfs[i], bm_df)


In [22]:
dfs['rrg']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
2293   -1.0
2294   -1.0
2295   -1.0
2296   -1.0
2297   -1.0
Name: rrg, Length: 2298, dtype: float64